# Predicting Baseball Game Outcomes

In [2]:
# LOAD IN PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections as c
%matplotlib inline
import itertools
from collections import Counter
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()

In [ ]:
# CLEANING BASEBALL DATA 
with open('atbat_table.csv','r') as in_file, open('edited_atbat_table.csv','w') as out_file:
    seen = set()
    for line in in_file:
        if line in seen: continue 

        seen.add(line)
        out_file.write(line)

with open('pitch_table.csv','r') as in_file, open('edited_pitch_table.csv','w') as out_file:
    seen = set()
    for line in in_file:
        if line in seen: continue 

        seen.add(line)
        out_file.write(line)

In [13]:
# LOAD IN DATA FROM CSV
bat = pd.read_csv('edited_atbat_table.csv')
pitch = pd.read_csv('edited_pitch_table.csv')

bat = bat[['retro_game_id','game_id','home_team_id','away_team_id', 
    'bat_home_id','pit_mlbid','bat_mlbid','pit_hand_cd','bat_hand_cd', 
    'event_outs_ct','event_tx','event_cd','battedball_cd']] 

bat.head()

,retro_game_id,game_id,home_team_id,away_team_id,bat_home_id,pit_mlbid,bat_mlbid,pit_hand_cd,bat_hand_cd,event_outs_ct,event_tx,event_cd,battedball_cd
0,OAK201509010,415615,oak,ana,0,592533,594777,R,L,1,Flyout,2,F
1,OAK201509010,415615,oak,ana,0,592533,545361,R,R,0,Single,20,G
2,OAK201509010,415615,oak,ana,0,592533,405395,R,R,1,Lineout,2,L
3,OAK201509010,415615,oak,ana,0,592533,461815,R,L,1,Flyout,2,F
4,OAK201509010,415615,oak,ana,1,533167,519299,R,L,1,Groundout,2,G


In [12]:
#CREATE DUMMY VARIABLES FOR PLATE APPEARANCE OUTCOMES
pa = list(bat.event_tx)
uniq_plate_outcome = set(pa)

def pa_outcomes(pa_outcomes, outcome):
   return outcome in pa_outcomes

for i in uniq_plate_outcome:
   bat[i] = bat.event_tx.apply(lambda x: pa_outcomes(x, i))

bat.head()

,retro_game_id,regseason_fl,game_id,home_team_id,away_team_id,bat_home_id,pit_mlbid,bat_mlbid,pit_hand_cd,bat_hand_cd,...,Triple Play,Strikeout - DP,Pop Out,Fan interference,Flyout,Fielders Choice,Walk,Single,Triple,Sac Bunt
0,OAK201509010,T,415615,oak,ana,0,592533,594777,R,L,...,False,False,False,False,True,False,False,False,False,False
1,OAK201509010,T,415615,oak,ana,0,592533,545361,R,R,...,False,False,False,False,False,False,False,True,False,False
2,OAK201509010,T,415615,oak,ana,0,592533,405395,R,R,...,False,False,False,False,False,False,False,False,False,False
3,OAK201509010,T,415615,oak,ana,0,592533,461815,R,L,...,False,False,False,False,True,False,False,False,False,False
4,OAK201509010,T,415615,oak,ana,1,533167,519299,R,L,...,False,False,False,False,False,False,False,False,False,False


In [14]:
pd.get_dummies(bat.pit_hand_cd)
pd.get_dummies(bat.bat_hand_cd)

,L,R
0,1,0
1,0,1
2,0,1
3,1,0
4,1,0
5,0,1
6,1,0
7,0,1
8,1,0
9,0,1


In [ ]:
#Drop NA Rows in PITCH
    # Gets rid of rows w/ null values
    pitch = pitch[pd.notnull(pitch['pitch_type'])]
    # Drops columns w/ insignificant pitch types (pitch_out, eephus, intent, etc.)
    pitch = pitch[pitch.pitch_type != 'AB']
    pitch = pitch[pitch.pitch_type != 'EP']
    pitch = pitch[pitch.pitch_type != 'UN']
    pitch = pitch[pitch.pitch_type != 'IN']